# Cambridge ML Commando Course

## Titanic: Survival Prediction with Decision Trees

_The problem we would like to solve is to determine if a Titanic's passenger would have survived, given age, passenger class, and sex. We will work with the Titanic dataset that can be downloaded from http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic.txt._

Start by importing numpy, scikit-learn, and pyplot, the Python libraries we will be using in this chapter. Show the versions we will be using (in case you have problems running the notebooks).

In [ ]:
import IPython
import sklearn as sk
import numpy as np
import pandas
import matplotlib
import matplotlib.pyplot as plt
import pydot
import pyparsing

print ('IPython version:', IPython.__version__)
print ('numpy version:', np.__version__)
print ('scikit-learn version:', sk.__version__)
print ('matplotlib version:', matplotlib.__version__)
#print ('pydot version:', pydotplus.__version__)
print ('pyparsing version:', pyparsing.__version__)

### Preprocessing

As usual, we should first load the dataset. The list of attributes it includes is: Ordinal, Class, Survived (0=no, 1=yes), Name, Age, Port of Embarkation, Home/Destination, Room, Ticket, Boat, and Sex. We will start by loading the dataset into a numpy array.. We assume it is located in the data/titanic.csv file

In [ ]:
df = pandas.read_csv("data/titanic.csv")
print(df[0:10])

df = df[["pclass","age","sex","survived"]]

mean_age = np.mean(df["age"])
df["age"] = df["age"].fillna(mean_age)

gender_enc = LabelEncoder()
df["sex"] = gender_enc.fit_transform(df["sex"])

class_enc = LabelEncoder()
df["pclass"] = class_enc.fit_transform(df["pclass"])
print("\nCLEANED----\n",df[0:10])

titanic_X = df[["pclass","age","sex"]]
titanic_y = df["survived"]

In [ ]:
# # import csv
# # with open('data/titanic.csv') as csvfile:
# #     titanic_reader = csv.reader(csvfile, delimiter=',', quotechar='"')
# #     # Header contains feature names
# #     row = next(titanic_reader)
# #     feature_names = np.array(row)
# #     # Load dataset, and target classes
# #     titanic_X, titanic_y = [], []
# #     for row in titanic_reader:  
# #         titanic_X.append(row)
# #         titanic_y.append(row[2]) # The target value is "survived"
# #     titanic_X = np.array(titanic_X)
# #     titanic_y = np.array(titanic_y)

# # Let's inspect how data looks. 

# print (feature_names,"\n", titanic_X[0:10],"\n", titanic_y[0:10],"\n")
# print (titanic_y[0:10])

# # We will keep only class (1st,2nd,3rd), age (float), and sex (male, female) for our study. 

# # we keep the class, the age and the sex
# titanic_X = titanic_X[:, [1, 4, 10]]
# feature_names = feature_names[[1, 4, 10]]
# print(feature_names)
# print(titanic_X[12], titanic_y[12])

# #We have some problems with missing values ('NA') for the 'age' feature. To avoid this, we will use the mean value whenever we do not have data available.

# ages = titanic_X[:, 1]
# mean_age = np.mean(titanic_X[ages != 'NA', 1].astype(np.float))
# titanic_X[titanic_X[:, 1] == 'NA', 1] = mean_age
# print(feature_names)
# print(titanic_X[12], titanic_y[12])

# ##Now, class and sex are categorical classes, but most scikit-learn classifiers (in particular the Decision Trees we plan to use), expect real-valued attributes. We can easily convert sex  to a binary value (0=female,1=male). We will use the LabelEncoder class from scikit-learn:

# from sklearn.preprocessing import LabelEncoder
# enc = LabelEncoder()
# # print(help(type(enc)))


# label_encoder = enc.fit(titanic_X[:, 2])
# print("Categorical classes:", label_encoder.classes_)
# integer_classes = label_encoder.transform(label_encoder.classes_)
# print ("Integer classes:", integer_classes)
# # t = label_encoder.transform(titanic_X[:, 2])
# # titanic_X[:, 2] = t
# print ('Feature names:',feature_names)
# print ('Features for instance number 12:',titanic_X[12], titanic_y[12])

# ##Now to convert the passenger class. Since we have three different categories, we cannot convert to binary values (and using 0,1,2 values would imply an order, something we do not want). We use OneHotEncoder to get three different attributes:

# from sklearn.preprocessing import OneHotEncoder

# #rjm49 - start by converting the pclass labels (column 0) into integer labels
# lab_enc = LabelEncoder()
# print(titanic_X[:,0])
# new_features = lab_enc.fit_transform(titanic_X[:, 0])
# print("unique new features", np.unique(new_features))

# print("Categorical classes:", label_encoder.classes_)
# integer_classes = label_encoder.transform(label_encoder.classes_)#.reshape(3, 1)
# print("Integer classes:", integer_classes.shape)

# #rjm49 - get the integer labels for all the passengers
# int_labels = label_encoder.transform(titanic_X[:,0]).reshape(-1,1)
# print("int_labels:\n", int_labels)

# #rjm49 - next, convert integer labels into "one hot" flags e.g. 0->[1 0 0], 1->[0 1 0], 2->[0 0 1]
# onehot_enc = OneHotEncoder()
# onehot_enc.fit(integer_classes.reshape(3, 1)) #rjm49 - note that this expects the classes to be passed in as a 3x1

# #create a sparse matrix with three columns, each one indicating if the instance belongs to the class
# new_features = onehot_enc.transform( int_labels ).toarray()
# print("new_features are:\n", new_features)

# print(titanic_X.shape)
# print(new_features.shape)
# titanic_X = np.concatenate([titanic_X, new_features.reshape(-1,1)], axis = 1) # rjm49, add the three new columns

# #Delete the old columns which have now been converted
# titanic_X = np.delete(titanic_X, [0], axis=1)
# # Update feature names
# feature_names = ['age', 'sex', 'class']# 'first_class', 'second_class', 'third_class']
# # Convert to numerical values
# titanic_X = titanic_X.astype(float)
# titanic_y = titanic_y.astype(float)

# print(titanic_X)

# print ('New feature names:',feature_names)
# print ('Values:',titanic_X[0])

Split into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(titanic_X, titanic_y, test_size=0.25, random_state=33)
print(X_train[0:10])
print(y_train[0:10])

### Decision Trees

Now, we can create a new DecisionTreeClassifier and use the fit method of the classifier to do the learning job.

In [ ]:
from sklearn import tree
clf_dt = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_leaf=5)
clf_dt = clf_dt.fit(X_train,y_train)
print(clf_dt)

The decision tree we have built represents a series of decisions based on the training data. To classify an instance, we should answer the question at each node. For example, at our root node, the question is: Is sex<=0.5? (are we talking about a woman?). If the answer is yes, you go to the left child node in the tree; otherwise you go to the right child node. You keep answering questions (was she in the third class?, was she in the first class?, and was she below 13 years old?), until you reach a leaf. When you are there, the prediction corresponds to the target class that has most instances (that is if the answers are given to the previous questions). In our case, if she was a woman from second class, the answer would be 1 (that is she survived), and so on. Let's drawit, using pyplot:

In [ ]:
import io
dot_data = io.StringIO() 
sk.tree.export_graphviz(clf_dt, out_file=dot_data, feature_names=titanic_X.columns, filled=True) 
(graph,) = pydot.graph_from_dot_data(dot_data.getvalue()) 
from IPython.core.display import Image 
Image(graph.create_png())

# sk.tree.plot_tree(clf_dt.fit(titanic_X, titanic_y))

Let's measure Accuracy, precision, recall, f1 in the training set

In [ ]:
from sklearn import metrics
def measure_performance(X,y,clf, show_accuracy=True, show_classification_report=True, show_confusion_matrix=True):
    y_pred=clf.predict(X)   
    if show_accuracy:
        print( "Accuracy:{0:.3f}".format(metrics.accuracy_score(y,y_pred)),"\n")

    if show_classification_report:
        print( "Classification report")
        print( metrics.classification_report(y,y_pred),"\n")
        
    if show_confusion_matrix:
        print( "Confusion matrix")
        print( metrics.confusion_matrix(y,y_pred),"\n")
        
measure_performance(X_train,y_train,clf, show_classification_report=False, show_confusion_matrix=False)

For this example, we will use an extreme case of cross- validation, named leave-one-out cross-validation. For each instance in the training sample, we train on the rest of the sample, and evaluate the model built on the only instance left out. After performing as many classifications as training instances, we calculate the accuracy simply as the proportion of times our method correctly predicted the class of the left-out instance, and found it is a little lower (as we expected) than the resubstitution accuracy on the training set.

In [ ]:
from sklearn.model_selection import cross_val_score, LeaveOneOut #model_selection replaces cross_validation
from scipy.stats.stats import sem

def loo_cv(X_train,y_train,clf):
    # Perform Leave-One-Out cross validation
    # We are performing 1313 classifications!
    loo = LeaveOneOut()
    scores=np.zeros(len(X_train)) #rjm49 - list of zeroes that's as long as X_train
    #for train_index,test_index in loo(X_train):
#     first = True
    y_hats = []
    for train_index,test_index in loo.split(X_train):
#         if first:
#             print(train_index, test_index)
#             first = False
        X_train_cv, X_test_cv= X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_cv, y_test_cv= y_train.iloc[train_index], y_train.iloc[test_index]
        clf = clf.fit(X_train_cv,y_train_cv)
        y_pred=clf.predict(X_test_cv)
        y_hats.append(int(y_pred))
#         print(y_test_cv.astype(int), y_pred.astype(int))
        scores[test_index]=sk.metrics.accuracy_score(y_test_cv.astype(int), y_pred.astype(int))
#         f1s[test_index]=sk.metrics.f1_score(y_test_cv.astype(int), y_pred.astype(int))
    f1 = sk.metrics.f1_score(y_train, y_hats)
    r = sk.metrics.recall_score(y_train, y_hats)
    p = sk.metrics.precision_score(y_train, y_hats)    
    from sklearn.metrics import confusion_matrix
    from scipy.stats import skew
    print("skew:",skew(y_train))
    print("Mean acc: {0:.3f} (+/-{1:.3f})".format(np.mean(scores), np.std(scores)))
    print("Macro p/r/f1: {:.3f}/{:.3f}/{:.3f}".format(p,r,f1))
    print(confusion_matrix(y_train, y_hats))


In [ ]:
print("Home cooked LOOCV:")
loo_cv(titanic_X, titanic_y, clf_dt)

print("\nUse sklearn's cross_val_score util:")
cv_scores = cross_val_score(clf_dt, titanic_X, titanic_y, cv=LeaveOneOut())#.split(titanic_X))
print(cv_scores.mean(), cv_scores.std())

### Random Forests

A common criticism to decision trees is that once the training set is divided after answering a question, it is not possible to reconsider this decision. For example, if we divide men and women, every subsequent question would be only about men or women, and the method could not consider another type of question (say, age less than a year, irrespective of the gender). Random Forests try to introduce some level of randomization in each step, proposing alternative trees and combining them to get the final prediction. These types of algorithms that consider several classifiers answering the same question are called ensemble methods. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(n_estimators=10, max_depth=5, min_samples_leaf=5)
clf_rf = clf_rf.fit(X_train,y_train)
loo_cv(X_train,y_train,clf_rf)

To evaluate performance on future data, evaluate on the training set and test on the evaluation set

In [ ]:
print("Decision tree:")
measure_performance(X_test,y_test,clf_dt)

print("Random forest:")
measure_performance(X_test,y_test,clf_rf)

#rjm49 -try your luck here... 
my_age = 19 #your age in Earth years
my_gender = 0 #[0, 1] for female, male
my_class = 0 #[0, 1, 2] for 1st, 2nd, 3rd
me = np.array([my_class, my_age, my_gender]).reshape(1,-1)

print("DTree thinks I'm {:.3f} likely to survive.".format(clf_dt.predict_proba(me)[0][1]))
print("RForest thinks I'm {:.3f} likely to survive.".format(clf_rf.predict_proba(me)[0][1]))
